- Heat map of difference in first and second component? Use abs difference and see if there is anything
- Look into l vs d component plot. Is this showing the rotation or have I found something unkown?
- 

In [1]:
%matplotlib widget
import numpy as np
#import aplpy as apl
#import matplotlib.colors as col
import matplotlib.pyplot as plt
import spectral_cube as sc
#from matplotlib.colorbar import ColorbarBase
from astropy.wcs import WCS
from astropy.io import fits
import time

In [3]:
def v2ch(v, w): # convert velocity (km/s) to channel 
    x_tempo, y_tempo, v_tempo   = w.wcs_pix2world(0, 0, 0, 0)
    x_ch, y_ch, v_ch   = w.wcs_world2pix(x_tempo, y_tempo, v*1000.0, 0)
    v_ch = int(round(float(v_ch), 0))
    return v_ch

def ch2v(ch, w):#km/s
    x, y, v   = w.wcs_pix2world(0, 0, ch, 0)
    return v/1000.0

def del_header_key(header, keys): # delete header key
    import copy
    h = copy.deepcopy(header)
    for k in keys:
        try:
            del h[k]
        except:
            pass
    return h

def make_new_hdu_integ(hdu, v_start_wcs, v_end_wcs, w): # make header of integrated intensity map
    data = hdu.data
    header = hdu.header
    start_ch, end_ch = v2ch(v_start_wcs, w), v2ch(v_end_wcs, w)
    new_data = np.sum(data[start_ch:end_ch+1], axis=0)*np.abs(header["CDELT3"])/1000.0
    header = del_header_key(header, ["CRVAL3", "CRPIX3", "CRVAL3", "CDELT3", "CUNIT3", "CTYPE3", "CROTA3", "NAXIS3", "PC1_3", "PC2_3", "PC3_3", "PC3_1", "PC3_2"])
    header["NAXIS"] = 2
    new_hdu = fits.PrimaryHDU(new_data, header)
    return new_hdu

def make_new_hdu_integ_ch(hdu, v_start_ch, v_end_ch, w): # make header of integrated intensity map
    data = hdu.data
    header = hdu.header
    new_data = np.sum(data[v_start_ch:v_end_ch], axis=0)*header["CDELT3"]/1000.0
    header = del_header_key(header, ["CRVAL3", "CRPIX3", "CRVAL3", "CDELT3", "CUNIT3", "CTYPE3", "CROTA3", "NAXIS3", "PC1_3", "PC2_3", "PC3_3", "PC3_1", "PC3_2"])
    header["NAXIS"] = 2
    new_hdu = fits.PrimaryHDU(new_data, header)
    return new_hdu

def new_header_v_decomp(original_hdu,max_fitted,val_type='unkown',val_unit='unkown'):
    header = original_hdu.header
    header["BTYPE"] = val_type
    header["BUNIT"] = val_unit
    #header["CTYPE1"] = val_type
    header["CUNIT3"] = val_unit
    header["NAXIS3"] = max_fitted
    header["CRVAL3"] = 1
    header["CDELT3"] = 1
    return header

In [4]:
species = '13CO'
filename = f"C:\\Users\\alexf\Japan internship\\{species}\\ngc1333TP.{species}.cube.valueK.fits"
hdu = fits.open(filename)[0]
original_header = hdu.header
w = WCS(hdu)

In [5]:
temp_params = []
temp_errors = []
temp_residuals = []
temp_model = []
temp_threshold = []
dec_cont=0
fit_count = np.ones((860,560)) * np.nan

# Load fit params intially to find number of fits per pixel
for num in range(6):
    temp = (np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\fitparams_chunk_{num}.npy',allow_pickle=True))
    if num!=0:
        dec_cont+= np.shape(temp)[0]

    print(dec_cont)
    for i in range(np.shape(temp)[0]):
        for j in range(np.shape(temp)[1]):
            fit_count[i+dec_cont,j] = np.shape(temp[i,j])[0] - 1
del temp                  # delete temp array

# Load each processed chunk and append arrays into one list
for num in range(6):
    temp_params.append (np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\fitparams_chunk_{num}.npy',allow_pickle=True))
    temp_errors.append (np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\fiterrors_chunk_{num}.npy',allow_pickle=True))
    #temp_residuals.append (np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\residuals_chunk_{num}.npy',allow_pickle=True))
    temp_threshold.append(np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\threshold_{num}.npy',allow_pickle=True))
    temp_model.append(np.load(f'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\model_chunk_{num}.npy',allow_pickle=True))

# Combine lists into one large array of correct size, delete temp arrays after for memory
stack_param = np.vstack(temp_params)
stack_errors = np.vstack(temp_errors)
#stack_residuals = np.vstack(temp_residuals)                               # Residuals is now created in this file instead of loaded in
stack_model = np.vstack(temp_model)
stack_threshold = np.vstack(temp_threshold)
del temp_params, temp_errors, temp_residuals, temp_threshold, temp_model

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\alexf\\Nova-NGC1333\\Nova-NGC1333\\output\\fitparams_chunk_0.npy'

In [5]:
max_fitted = int(np.nanmax(fit_count))

# Actual arrays to be used. Shaped dec X ra X fit number
print('Creating arrays...')
start_time = time.time()
fit_velocity = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
fit_scaling = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
fit_dispersion = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
err_velocity = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
err_scaling = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
err_dispersion = np.ones((860,560,max_fitted),dtype=np.float32) * np.nan
model = np.ones((860,560,max_fitted,len(stack_model[0,0][0])),dtype=np.float16) * np.nan
#final_residuals = np.ones((860,560,len(stack_model[0,0][0])),dtype=np.float32) * np.nan

final_model = np.ones((860,560,len(stack_model[0,0][0])),dtype=np.float32) * np.nan
threshold = np.ones((860,560,max_fitted)) * np.nan
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")


print('Assigning values...')
start_time = time.time()

# Loop through each ra and dec combination then each fit for that positions
# Doing it this way gets around any problems form inhomogenous arrays as that was needed based on not knowing how many fits would be needed
for i in range(860):
    print(i)
    for j in range(560):
        #final_residuals[i,j] = stack_residuals[i,j]
        threshold[i,j] = stack_threshold[i,j]
        for k in range(max_fitted):
            try:
                fit_scaling[i,j,k] = stack_param[i,j][k][0]
                fit_velocity[i,j,k] = stack_param[i,j][k][1]
                fit_dispersion[i,j,k] = stack_param[i,j][k][2]
                
                err_scaling[i,j,k] = stack_errors[i,j][k][0]
                err_velocity[i,j,k] = stack_errors[i,j][k][1]
                err_dispersion[i,j,k] = stack_errors[i,j][k][2]
                
                model[i,j,k] = stack_model[i,j][k]
                
            except IndexError:
                # Looping up to the max number of fits, not all of them have an index that high
                continue
            
            finally:
                # Create the model data. temp_model is only used here to make the code a bit easier to follow
                model[i,j][model[i,j]==-320]=0
                # Sum each fitted gaussian together
                final_model[i,j] = np.nansum(model[i,j],axis=0)
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")

# Transpose the array so that it matches the ALMA data (channel x dec x ra)
print('Creating residuals...')
start_time = time.time()
final_model = np.transpose(final_model,axes=[2,0,1])
final_residuals = hdu.data- final_model
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")



print('Replacing placeholder with nan...') 
# Need to do it like this due to inhomogenous data types earlier. Python needs those to be 'pickled' to save them but that can corrupt data if nan values are present
# -320 is the placeholder set before saving so this just swaps it back
start_time = time.time()
fit_scaling[fit_scaling==-320]=np.nan  
fit_velocity[fit_velocity==-320]=np.nan
fit_dispersion[fit_dispersion==-320]=np.nan

err_scaling[err_scaling==-320]=np.nan
err_velocity[err_velocity==-320]=np.nan
err_dispersion[err_dispersion==-320]=np.nan

final_residuals[final_residuals==-320]=np.nan
threshold[threshold==-320]=np.nan 
#model[model==-320]=0
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")



print('Stacking values for saving...')
start_time = time.time()
stacked_total_vals = np.stack([fit_scaling,fit_velocity,fit_dispersion,err_scaling,err_velocity,err_dispersion])
del fit_scaling, fit_velocity, fit_dispersion, err_scaling, err_velocity, err_dispersion
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")


print('Creating parameter cubes and saving...')
# Loop for each value type and again for each fit type. Create a new header with the correct wcs and dimensions and then save it as a cube
start_time = time.time()
fit_value = ['Scaling','Velocity','Dispersion']
fit_type = ['Fit','Error']
chan_num = '10chan'
count=0
for types in fit_type:
    for values in fit_value:
        current_data = stacked_total_vals[count]
        new_header = new_header_v_decomp(hdu,max_fitted,val_type=f"{values}",val_unit="km/s")
        trans = np.transpose(current_data,axes=[2,0,1])
        new_cube = sc.SpectralCube(data=trans,header=new_header,wcs=w)
        new_cube.write(f'V_decomp//C18O//{species}_{types}_{values}.{chan_num}.cube.fits',format='fits',overwrite=True)
        count+=1
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")
del new_cube, stack_errors, stack_param, stacked_total_vals, fit_value, fit_type    # Delete variables now they are saved as large cubes can be expensive


print('Creating residual cube, model cube and saving...')
start_time = time.time()
new_header = new_header_v_decomp(hdu,len(stack_model[0,0][0]),val_type='Residuals',val_unit='km/s')
new_cube = sc.SpectralCube(data=final_residuals,header=original_header,wcs=w)
new_cube.write(f'V_decomp//C18O//{species}_residuals.{chan_num}.cube.fits',format='fits',overwrite=True)

# Need to mask the total model as anything outside of the observed field is also set to 0
mask = np.isfinite(final_residuals)
new_cube = sc.SpectralCube(data=final_model,header=original_header,wcs=w)
new_cube = new_cube.with_mask(mask)
new_cube.write(f'V_decomp//C18O//{species}_total_model.{chan_num}.cube.fits',format='fits',overwrite=True)

# Save the non summed model. Mainly for debugigng to see which gaussians were used to create each point of the model
np.save('V_decomp//C18O//non_summed-model',model)
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")


print('Saving number of gaussian and threshold plot...')
start_time = time.time()
mask_single = np.isfinite(final_residuals[0,:,:])
fit_count_masked = np.where(mask_single==True,fit_count,np.nan)
fit_map_fits = fits.PrimaryHDU(data=fit_count_masked)
fit_map_fits.writeto(f'V_decomp//C18O//{species}_fit_count.{chan_num}.fits',overwrite=True)

trans = np.transpose(threshold,axes=[2,0,1])
threshold_map = fits.PrimaryHDU(data=trans)
threshold_map.writeto(f'V_decomp//C18O//{species}_threshold.{chan_num}.fits',overwrite=True)
print(f"Execution time: { (time.time() - start_time) /60:.2f} minutes")
print("Done :)")

Creating arrays...
Execution time: 1.59 minutes
Assigning values...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
26

In [6]:
%reset -f

In [24]:
# Old v_decomp version based on multi-threading
# Not with the other due to it being a .py so no cells
'''
# Make optmize warning into errors (i.e. covariance can't be estimated so it is a poorfit
import warnings
warnings.filterwarnings('error','Covariance of the parameters could not be estimated',scipy.optimize._optimize.OptimizeWarning)
def gaussian(velocity,A,v_0, sigma):
    return (A * np.exp( - (velocity-v_0)**2 / (2*sigma**2)))


def gaussian_to_max(ra,dec,current_slice,index=0):
    # Find V_rad channel at max T
    try:
        channel_max = np.nanargmax(current_slice)
    except ValueError:
       # print(f"Ra index: {ra}, Dec index: {dec} is all Nans. Skipping...")
        return (np.nan,np.nan,np.nan)
    
    if channel_max <= 10:
        print(f"Max before channel 10: Ra = {ra}, Dec = {dec} - Thread {index}")
        return (np.nan,np.nan,np.nan)
    # Fit gaussian to channel
    try:
        velocity_current_cont = velocity_cont[channel_max-10:channel_max+10]
        selected_current_slice = current_slice[channel_max-10:channel_max+10]
        
        if (len(np.isfinite(selected_current_slice))<=4):
            print(f"Fewer than 5 data points: Ra = {ra}, Dec = {dec} - Thread {index}")
            return (np.nan,np.nan,np.nan)

        pop, pcov = curve_fit(gaussian,xdata=velocity_current_cont,ydata=selected_current_slice,p0=[5, velocity_cont[channel_max],20],nan_policy='omit',bounds=([0,-85,0],[1000,95,100]))
    except RuntimeError:
        print(f"Runtime error: Ra = {ra}, Dec = {dec} - Thread {index}")
        return (np.nan,np.nan,np.nan)
    except IndexError:
        print(f"Index error: Ra = {ra}, Dec = {dec} - Thread {index}")
        return (np.nan,np.nan,np.nan)
    except scipy.optimize._optimize.OptimizeWarning:
        print(f"Inditerminate covariance: Ra = {ra}, Dec = {dec} - Thread {index}")
        return (np.nan,np.nan,np.nan)
    
    # Construct continuus gaussian and subtract from data 
    fitted_gauss = gaussian(velocity_cont,pop[0],pop[1],pop[2])
    residuals = current_slice - fitted_gauss
    errors = np.sqrt(np.diag(pcov))
    return (residuals,pop,errors)
    
    # Chat GPT version, lists instead of arrays to solve appending issue

def process_chunk(data, index=0, dec_start_local=0):
    # Initialize lists instead of arrays
    residuals_local = []
    fit_params_local = []
    fit_errors_local = []
    
    shape = np.shape(data)
    ra_len = shape[2]
    dec_len = shape[1]
    
    for ra in range(ra_len):
        dec_pixel = dec_start_local
        print(f'Processing RA: {ra}/{ra_len}, dec length: {dec_len} - Thread {index}')
        
        for dec in range(dec_len):
            
            for i in range(2):
                if i == 0:
                    current_slice = data[:, dec, ra]
                    current_residuals = np.zeros_like(current_slice)
                else:
                    current_slice = current_residuals
                    
                current_slice = np.where(current_slice >= threshold, current_slice, np.nan)
                current_residuals, fit_param, fit_error = gaussian_to_max(ra + 1, dec_pixel + 1, current_slice, index)
                
                if np.isnan(current_residuals).all():
                    current_residuals = current_slice
                    fit_param, fit_error = (np.nan, np.nan, np.nan), (np.nan, np.nan, np.nan)
                else:
                    print(f"Ra pixel: {ra + 1}, Dec pixel {dec_pixel + 1} worked. V_0 = {fit_param[1]} - Thread {index}")
                
                # Append data to lists
                residuals_local.append(current_residuals)
                fit_params_local.append(fit_param)
                fit_errors_local.append(fit_error)
                
            dec_pixel += 1
        
    print(f"Thread {index} finished :)")
    
    # Convert lists to arrays
    residuals_local = np.array(residuals_local)
    fit_params_local = np.array(fit_params_local)
    fit_errors_local = np.array(fit_errors_local)
    
    np.save(f'output/residuals_chunk_{index}',residuals_local)
    np.save(f'output/fitparams_chunk_{index}',fit_params_local)
    np.save(f'output/fiterrors_chunk_{index}',fit_errors_local)
    
    return 0

chunks = np.array_split(data,6,axis=1)
print(np.shape(chunks[0]))

ra_start, dec_start = [],[]
previous_dec_start = 0
for i in range(6):
    if i!= 0:
        shape_current = np.shape(chunks[i])
        current_dec_start = shape_current[1]
        dec_start = np.append(dec_start,current_dec_start+previous_dec_start)
        
        previous_dec_start = current_dec_start + previous_dec_start
        
    else:
        dec_start = np.append(dec_start,0)

#thread_1 = threading.Thread(target=process_chunk, args = (chunks[0],1,dec_start[0]))
#thread_2 = threading.Thread(target=process_chunk, args = (chunks[1],2,dec_start[1]))
#thread_3 = threading.Thread(target=process_chunk, args = (chunks[2],3,dec_start[2]))
#thread_4 = threading.Thread(target=process_chunk, args = (chunks[3],4,dec_start[3]))
#thread_5 = threading.Thread(target=process_chunk, args = (chunks[4],5,dec_start[4]))
#thread_6 = threading.Thread(target=process_chunk, args = (chunks[5],6,dec_start[5]))


#thread_1_vals = thread_1.start()
#thread_2_vals = thread_2.start()
#thread_3_vals = thread_3.start()
#thread_4_vals = thread_4.start()
#thread_5_vals = thread_5.start()
#thread_6_vals = thread_6.start()  '''  

'\n# Make optmize warning into errors (i.e. covariance can\'t be estimated so it is a poorfit\nimport warnings\nwarnings.filterwarnings(\'error\',\'Covariance of the parameters could not be estimated\',scipy.optimize._optimize.OptimizeWarning)\ndef gaussian(velocity,A,v_0, sigma):\n    return (A * np.exp( - (velocity-v_0)**2 / (2*sigma**2)))\n\n\ndef gaussian_to_max(ra,dec,current_slice,index=0):\n    # Find V_rad channel at max T\n    try:\n        channel_max = np.nanargmax(current_slice)\n    except ValueError:\n       # print(f"Ra index: {ra}, Dec index: {dec} is all Nans. Skipping...")\n        return (np.nan,np.nan,np.nan)\n    \n    if channel_max <= 10:\n        print(f"Max before channel 10: Ra = {ra}, Dec = {dec} - Thread {index}")\n        return (np.nan,np.nan,np.nan)\n    # Fit gaussian to channel\n    try:\n        velocity_current_cont = velocity_cont[channel_max-10:channel_max+10]\n        selected_current_slice = current_slice[channel_max-10:channel_max+10]\n        \n